#### Lesson 3: Reflection and Blogpost Writing

#### Setup

In [1]:
#r "nuget:AutoGen,0.1.0"
#load "util.csx"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIModel = "gpt-4o-mini";
var openaiClient = OpenAIClientProvider.Create();

Installed Packages AutoGen, 0.1.0

The Task!

In [2]:
var task = """
    Write a concise but engaging blogpost about
    DeepLearning.AI. Make sure the blogpost is
    within 100 words.
    """;

Create a writer agent

In [3]:
var writer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Writer",
    modelName: openAIModel,
    systemMessage: """
    You are a writer. You write engaging and concise blogpost (with title) on given topics.
    You must polish your writing based on the feedback you receive and give a refined version.
    Only return your final work without additional comments."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

var reply = await writer.SendAsync(task);

from: Writer
**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of the artificial intelligence revolution, empowering individuals and organizations with cutting-edge resources and education. Founded by AI pioneer Andrew Ng, the platform offers comprehensive courses that demystify complex concepts in machine learning and deep learning. With a robust community and practical applications, learners can bridge the gap between theory and real-world implementation. Whether you're a novice seeking foundational knowledge or a professional looking to refine your skills, DeepLearning.AI provides the tools and expertise to harness the transformative power of AI. Join the movement and shape the future of technology!



Adding reflection by creating a critic agent to reflect on the work of the writer agent.

In [4]:
var critic = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Critic",
    modelName: openAIModel,
    systemMessage: """
    You are a critic. You review the work of the writer and provide constructive feedback to help improve the quality of the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

var conversation = await critic.SendAsync(
    receiver: writer,
    message: task,
    maxRound: 3)
    .ToListAsync();

// Use the last message in the conversation as the final result
var res = conversation.Last();

from: Writer
**Unlocking AI Potential: DeepLearning.AI**

DeepLearning.AI is at the forefront of the artificial intelligence revolution, empowering individuals and organizations through innovative education and resources. Founded by Andrew Ng, this platform offers comprehensive courses that demystify deep learning and machine learning concepts, making them accessible to all. Whether you’re a beginner or a seasoned professional, DeepLearning.AI provides the tools and expertise needed to harness the power of AI. With a commitment to fostering a global community of AI practitioners, it inspires learners to tackle real-world challenges. Dive into DeepLearning.AI and unlock your potential in the ever-evolving tech landscape!

from: Critic
**Feedback:**

Your blog post effectively captures the essence of DeepLearning.AI in a concise manner. The introduction is engaging, and the mention of Andrew Ng adds credibility. However, consider the following suggestions to enhance clarity and impact:



#### Nested chat using middleware pattern
Python AutoGen allows you to create a Json object task list and pass it to agent using `register_nested_chats`
This is not supported in AutoGen.Net.
But you can achieve the same using the middleware pattern.

In [5]:
var SEOReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "SEO_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are an SEO reviewer, known for your ability to optimize content for search engines, ensuring that it ranks well and attracts organic traffic.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [6]:
var LegalReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Legal_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are a legal reviewer, known for your ability to ensure that content is legally compliant and free from any potential legal issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [7]:
var EthicsReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Ethics_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are an ethics reviewer, known for your ability to ensure that content is ethically sound and free from any potential ethical issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [8]:
var MetaReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Meta_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are a meta reviewer, you aggragate and review the work of other reviewers and give a final suggestion on the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

Define nest chat middleware

In [10]:
class NestChatMiddleware : IMiddleware
{
    private readonly IAgent seo;
    private readonly IAgent legal;
    private readonly IAgent ethics;
    private readonly IAgent meta;
    private readonly IAgent writer;

    public NestChatMiddleware(IAgent writer, IAgent seo, IAgent legal, IAgent ethics, IAgent meta)
    {
        this.writer = writer;
        this.seo = seo;
        this.legal = legal;
        this.ethics = ethics;
        this.meta = meta;
    }

    public string? Name => nameof(NestChatMiddleware);

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent critic, CancellationToken cancellationToken = default)
    {
        // trigger only when the last message is from writer
        if (context.Messages.Last().From != writer.Name)
        {
            return await critic.GenerateReplyAsync(context.Messages, context.Options, cancellationToken);
        }

        var messageToReview = context.Messages.Last();
        var reviewPrompt = $"""
            Review the following content.

            {messageToReview.GetContent()}
            """;
        // SEO Review
        var seoReview = await critic.SendAsync(
            receiver: seo,
            message: reviewPrompt,
            maxRound: 1)
            .ToListAsync();
        var legalReview = await critic.SendAsync(
            receiver: legal,
            message: reviewPrompt,
            maxRound: 1)
            .ToListAsync();

        var ethicsReview = await critic.SendAsync(
            receiver: ethics,
            message: reviewPrompt,
            maxRound: 1)
            .ToListAsync();

        var reviews = seoReview.Concat(legalReview).Concat(ethicsReview);

        var metaReview = await critic.SendAsync(
            receiver: meta,
            message: "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
            chatHistory: reviews,
            maxRound: 1)
            .ToListAsync();

        var lastReview = metaReview.Last();
        lastReview.From = critic.Name;

        return lastReview;
    }
}

In [11]:
// Orchestrate the nested chats to solve the task
var middleware = new NestChatMiddleware(
    writer: writer,
    seo: SEOReviewer,
    legal: LegalReviewer,
    ethics: EthicsReviewer,
    meta: MetaReviewer);

In [12]:
var nestChatCritic = critic
    .RegisterMiddleware(middleware)
    .RegisterPrintMessage();

In [14]:
conversation = await nestChatCritic.SendAsync(
    message: task,
    receiver: writer,
    maxRound: 3)
    .ToListAsync();

var finalResult = conversation.Last();

from: Writer
**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering individuals with the skills needed to thrive in the AI landscape. Founded by Andrew Ng, the organization offers a range of online courses that make deep learning accessible to everyone, from beginners to seasoned professionals. Through hands-on projects and a robust community, learners gain practical experience while building their knowledge. As AI continues to transform industries, DeepLearning.AI stands as a vital resource, equipping the next generation of innovators to harness the potential of this groundbreaking technology. Join the movement and unlock your AI potential today!

from: SEO_Reviewer
As an SEO reviewer, here are my suggestions for optimizing the content:

- **Incorporate Keywords**: Include relevant keywords such as "AI courses," "deep learning training," and "artificial intelligence education" to enhance search visibility a